In [1]:
import pickle
import sqlite3
import numpy as np
import pandas as pd
execfile('Imports.py')
import Modules.Funcs as funcs
from Modules.Classes import Simulation
from Modules.Classes import CopyTweak
from Modules.Classes import Packer
from Modules.Classes import ConjugateJK13
from Modules.Classes import RepresentJK13
from Modules.Classes import CopyTweakRep
from Modules.Classes import PackerRep

dbname = 'experiments-5con.db'#raw data
dataname = '5con' #for parms
execfile('validate_data.py')

con = sqlite3.connect(dbname)
participants = pd.read_sql_query("SELECT participant, condition from participants", con)
generation = pd.read_sql_query("SELECT * from generation", con)
alphas = pd.read_sql_query("SELECT * from alphas", con)
stimuli = pd.read_sql_query("SELECT * from stimuli", con).values
con.close()

#Get current best parms                                                                                                                    
with open(pickledir+bestparmchtc,'rb') as f:
    best_params_t = pickle.load(f)
best_params = funcs.compress_chtc_parms(best_params_t)




In [2]:
import matplotlib.pyplot as plt
wrap_ax = None
model = RepresentJK13#
params = best_params[model.model]
print(params)
c1 = np.array([[-1.,-1.],[1.,-1.],[-1.,1.],[1.,1.]])
c1 = np.array([[1.,1],[.75,1]])


c2 = np.array([[-.25,-.25],[-.5,-.5],[-.25,-.5],[1,-1]])
c3 = np.array([[0,0]])
categories = [c1,c2,c3]
catstr = ['Generating Alphas','Generating Betas','Generating Gammas']

STAT_LIMS = (-1.0, 1.0)
betacol = ['green' for bi in range(len(c2))]
f,ax = plt.subplots(1,len(categories),figsize=(3*len(categories),3))
# params['category_mean_bias'] = .1
for ci in range(len(categories)):
    ps = model(categories,params,wrap_ax=wrap_ax).get_generation_ps(stimuli,ci,'generate')
#     print(np.round(np.reshape(ps,(9,9)),3))
    gps = funcs.gradientroll(ps,'roll')[:,:,0]
    ps_ElRange = gps.max()-gps.min();
    plotVals = (gps-gps.min())/ps_ElRange
    if len(categories)>2:
        gammas = categories[2]
    else:
        gammas = []
    funcs.plotgradient(ax[ci], plotVals, categories[0], categories[1], gammas = gammas,clim = STAT_LIMS, cmap = 'PuOr',beta_col=betacol)
    ax[ci].set_title(catstr[ci])
# plt.close()
# categories2 = [np.array([[-1.,-1.],[1.,-1.],[-1.,-.75],[1.,-.75]])]
# temp2 = model(categories2,params,wrap_ax=wrap_ax).get_generation_ps(stimuli,1,'generate',wrap_ax=wrap_ax)
# print(np.round(np.reshape(temp2,(9,9)),3))

{'category_variance_bias': 1.0000000001078726, 'determinism': 2.452535524129651, 'category_mean_bias': 6.731660992152807, 'domain_variance_bias': 1.8971618173988867}


In [3]:
1.0==1
type(1) == int
a = None
not type(1.0) is int or not a == None

True

In [4]:
import pickle
with open('pickles/corner.p','rb') as f:
    trials2 = pickle.load(f)
trials2.task='generate'

In [13]:
temp = Simulation.extractPptData(trials2,'all',0)
temp.Set

[{'categories': [array([ 0,  8, 72, 80]), array([], dtype=float64)],
  'participant': [array([], dtype=int64),
   array([  9,  10,  11,  12,  14,  15,  16,  17,  20,  21,  22,  24,  25,
           26,  27,  29,  30,  32,  33,  37,  38,  39,  40,  41,  42,  44,
           45,  46,  52,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
           69,  71,  74,  78,  79,  80,  81,  82,  83,  85,  86,  88,  89,
           91,  92,  93,  94,  95,  97,  98,  99, 100, 101, 102, 103, 104,
          105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
          119, 120, 121, 122, 123, 124, 126, 128, 129, 130, 131])],
  'response': [array([], dtype=int64),
   array([67, 57,  5, 18, 48, 40, 40, 39, 39, 30, 39, 38, 42, 60, 38, 39, 72,
          40, 76, 66,  8, 20, 49, 49, 43, 20, 38,  0, 27, 38, 22, 22, 43,  2,
          31, 45, 76, 80,  3, 72, 40, 67, 39, 24, 66,  0, 40, 37, 53, 19, 64,
          70, 11,  6, 40, 40, 56, 22, 38, 67, 71, 69, 55,  3, 60, 59, 60, 58,
          39, 34, 39

In [119]:
import pickle
with open('pickles/corner_c.p','rb') as f:
    trials = pickle.load(f)

In [120]:
trials.Set

[{'categories': [array([ 0,  8, 72, 80]), array([], dtype=float64)],
  'participant': [array([], dtype=int64),
   array([ 10,  17,  20,  21,  22,  26,  27,  32,  33,  44,  45,  58,  59,
           60,  61,  62,  63,  64,  74,  78,  79,  80,  82,  83,  89,  92,
           93,  94,  95, 101, 104, 106, 107, 109, 110, 111, 115, 118, 119,
          120, 123, 128, 130])],
  'response': [array([], dtype=int64),
   array([57, 39, 39, 30, 39, 60, 38, 40, 76, 20, 38, 38, 22, 22, 43,  2, 31,
          45, 67, 39, 24, 66, 40, 37, 70,  6, 40, 40, 56, 69, 60, 60, 58, 39,
          34, 39,  2, 19, 41, 70, 30, 41, 38])],
  'wrap_ax': [array([], dtype=int64),
   array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]},
 {'categories': [array([ 0,  8, 72, 80]), array([57])],
  'participant': [array([], dtype=int64), array([10])],
  'response': [array([], dtype=int64), array([69])],
  'wrap_ax': [array([], dtype=i

In [61]:
for trial in trials.Set:
    if len(trial['participant'])>3:
        print trial

{'participant': [[], [259], [286, 402], []], 'response': [[], [274], [2489, 1299], []], 'categories': [array([ 403,  818, 1479, 1890]), array([   0,   49, 2450, 2499])], 'wrap_ax': [[], [None], [None, None], []]}


In [113]:
ps_add = np.array([])
ps_add = np.append(ps_add,[34,34,2])
print(ps_add)
np.array(ps_add,dtype=np.int16)

[34. 34.  2.]


array([34, 34,  2], dtype=int16)

In [102]:
len(np.where(np.unique([2,3,2,2])>3)[0])

0